In [ ]:
import sys
at_colab = "google.colab" in sys.modules
if 'google.colab' in sys.modules:
    import os
    from google.colab import files
    # just check if we already uploaded, may we restart the runtime and run all cells
    if not os.path.isfile('Data Set Feedcalculator.xlsx'):
        uploaded = files.upload()

Saving Data Set Feedcalculator.xlsx to Data Set Feedcalculator.xlsx


In [ ]:
if at_colab:
    import shutil
    if not shutil.which("pyomo"):
        !pip install -q pyomo
        assert(shutil.which("pyomo"))

     |████████████████████████████████| 9.1 MB 21.2 MB/s 
     |████████████████████████████████| 49 kB 7.0 MB/s 


In [ ]:
if at_colab:
    if not shutil.which('/usr/bin/glpsol'):
        !sudo apt install libglpk-dev python3.8-dev libgmp3-dev
        !apt-get install -y -qq glpk-utils
        assert(shutil.which('/usr/bin/glpsol'))

    if not shutil.which('/usr/bin/cbc'):
        !apt-get install -y -qq coinor-cbc
        assert(shutil.which('/usr/bin/cbc'))

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3
  libglpk40 libgmp-dev libgmpxx4ldbl libgraphblas1 libklu1 libldl2 libmetis5
  libpython3.8 libpython3.8-dev libpython3.8-minimal libpython3.8-stdlib
  librbio2 libspqr2 libsuitesparse-dev libsuitesparseconfig5 libumfpack5
  python3.8 python3.8-minimal
Suggested packages:
  libiodbc2-dev gmp-doc libgmp10-doc libmpfr-dev python3.8-venv python3.8-doc
  binfmt-support
The following NEW packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3
  libglpk-dev libglpk40 libgmp-dev libgmp3-dev libgmpxx4ldbl libgraphblas1
  libklu1 libldl2 libmetis5 libpython3.8 libpython3.8-dev libpython3.8-minimal
  libpython3.8-stdlib librbio2 libspqr2 libsuitesparse-dev
  libsuitesparseconfig5 libumfpack5 python3.8 python3.8-dev python3.8-mini

In [ ]:
import pyomo.environ as pyo
import pandas as pd
from io import StringIO
import numpy as np

In [ ]:
data = pd.read_excel('Data Set Feedcalculator.xlsx',sheet_name=None)

In [ ]:
ingredients      = data['Ingredient Database']
nutrient_rules   = data['Nutrient Rules']
ingredient_rules = data['Ingredient Rules']
ingredients


,Name,Reference name,Availability,Price,dm,moist,ash,cp,cfat,cfibre,staew,sug,cho,ca,p,ip,k,na,cl,oebr,oelh,opp,lys,met,cys,mc,thr,trp,ile,arg,phe,his,leu,tyr,phetyr,val,dlysp,dmetp,dcysp,dmcp,dthrp,dtryp,dilep,dargp,dvalp,dneaa,c182,c20,ne2015mj,ne2015kcal,ewcvb2015,vppi,sidlys,sidmet,sidcys,sidmc,sidthr,sidtrp,sidile,sidneaa,decat,pcat,alyscat,ametcat,acyscat,amccat,athrcat,atrpcat,ailecat,aargcat,aphecat,ahiscat,aleucat,atyrcat,aptcat,avalcat
0,Barley,barley,True,0.26,867.0,133.0,19.9,99.7,27.0,43.4,539.3,22.5,0.0,0.4,3.1,2.3,4.9,0.1,1.0,2656.5,2862.0,1.2,3.6,1.7,2.2,3.9,3.4,1.2,3.5,4.9,0.0,0.0,0.0,0.0,0.0,4.9,2.3,1.3,1.6,2.9,2.3,0.9,2.5,3.9,3.6,41.0,6.8,0.0,10.0,2389.5,1.1,1.0,2.7,1.4,1.7,3.1,2.7,1.0,2.9,44.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Blood meal 80% CP,blood,True,0.71,890.0,110.0,25.0,800.0,10.0,10.0,0.0,0.0,80.0,0.5,1.7,0.0,2.8,5.8,3.4,2450.0,2450.0,1.3,71.2,9.6,9.6,19.2,35.2,12.0,9.6,34.4,55.2,51.2,102.5,23.2,78.5,68.8,51.3,6.9,6.9,13.8,25.4,8.7,6.9,24.8,49.6,239.8,0.0,0.0,7.2,1716.4,0.8,1.0,60.0,7.6,7.6,15.3,28.0,9.8,6.5,263.6,3690.0,1.0,59.6,7.5,7.4,14.9,27.6,9.7,6.2,29.1,45.7,43.3,85.8,18.2,63.9,57.0
2,Blood meal 40% CP,bloodlq,False,0.55,930.0,70.0,500.0,400.0,10.0,0.0,0.0,0.0,20.0,1.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.6,4.8,4.8,9.6,17.6,6.0,4.8,17.2,27.6,25.6,51.2,11.6,39.2,34.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1527.9,1.5,24.2,3.1,3.0,6.1,11.2,3.9,2.5,11.8,18.5,17.6,34.8,7.4,25.9,26.3
3,Bone ash,boneash,True,0.18,954.0,46.0,806.1,0.0,0.0,0.0,0.0,0.0,0.0,289.3,133.8,0.0,0.4,9.5,0.0,0.0,0.0,82.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,87.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,107.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Brewers grain (dried),brew,False,0.08,915.0,85.0,46.0,248.0,70.0,132.0,40.0,9.0,551.0,3.5,4.6,3.0,0.5,0.1,0.1,0.0,0.0,0.0,9.4,4.7,4.7,9.4,9.2,3.0,9.9,12.1,12.4,5.7,19.6,7.7,20.1,13.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1340.7,2.8,5.3,2.9,2.5,5.4,5.1,1.7,6.0,8.1,7.9,3.4,12.0,5.1,13.0,7.9
5,Copra,copra,False,0.16,907.0,93.0,60.8,203.2,89.0,113.4,13.6,100.7,0.0,0.8,5.5,2.7,21.2,0.6,6.2,1889.0,1889.3,2.6,5.1,3.1,3.1,6.2,6.1,1.5,6.5,22.2,0.0,0.0,0.0,0.0,0.0,9.8,2.4,2.4,1.6,4.1,3.6,1.0,4.9,19.0,7.5,60.0,1.3,0.0,9.1,2163.2,1.0,1.3,2.9,1.8,1.8,3.6,3.5,0.8,3.8,49.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Cotton seed cake,cotton,True,0.20,933.0,67.0,59.7,362.9,79.0,169.8,33.6,38.3,0.0,2.2,10.2,7.7,14.4,0.0,0.3,1730.0,1890.3,3.1,14.9,5.8,6.2,11.9,11.7,4.4,11.3,38.8,0.0,0.0,0.0,0.0,0.0,16.0,8.8,3.9,4.2,8.1,8.0,3.2,8.1,32.7,11.8,125.4,28.4,0.6,6.8,1633.7,0.8,2.9,10.4,4.7,4.9,9.6,8.9,3.5,9.0,139.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Fats/ Vegetable oil,fats,False,1.50,995.0,5.0,0.0,0.0,995.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7230.0,9770.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500.0,35.0,31.8,7600.0,3.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8785.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Fish meal 56% CP,fish,True,0.63,911.0,89.0,247.0,563.0,90.0,0.0,0.0,0.0,11.0,60.0,39.6,0.0,6.4,10.5,15.2,2862.0,2982.0,19.6,42.8,15.8,5.1,20.9,23.6,6.2,23.6,33.2,22.0,14.6,41.1,17.4,36.4,27.6,38.5,14.3,4.5,18.8,20.1,5.6,21.0,30.6,32.3,211.2,1.3,56.9,8.8,2099.0,1.0,30.5,38.2,14.0,3.7,17.7,20.9,5.3,21.3,210.8,3501.0,23.8,36.4,13.3,4.2,17.5,19.6,5.0,19.6,28.8,18.3,12.1,34.5,14.3,32.6,22.7
9,Fish meal 40% CP,fishlq,True,0.55,900.0,100.0,399.0,384.0,100.0,0.0,0.0,0.0,18.0,141.0,67.0,0.0,6.4,10.5,15.2,0.0,0.0,0.0,24.8,7.6,2.6,10.2,12.2,2.8,9.9,20.8,10.6,6.0,17.9,7.1,17.8,12.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2626.6,32.8,20.7,6.0,2.1,8.1,10.3,2.3,8.3,17.3,8.4,4.6,13.8,6.2,14.6,9.4


In [ ]:
ingredient_bounds = { ing : (lb if not pd.isna(lb) else 0,    # no lower bound translates into 0 as lower bound
                             ub if not pd.isna(ub) else None) # no upper bound becomes None
                      for ing,lb,ub in zip(ingredient_rules.Ingredient,ingredient_rules['Lower Bound'],ingredient_rules['Upper Bound']) }

In [ ]:
ingredient_bounds_df = pd.DataFrame.from_dict(ingredient_bounds, orient='index')
ingredient_bounds_df = ingredient_bounds_df.set_axis(['Lower Bound', 'Upper Bound'], axis=1)
ingredient_bounds_df

,Lower Bound,Upper Bound
barley,0.0,0.100
blood,0.0,0.030
boneash,0.0,0.030
mbmeal,0.0,0.050
mbmeal2,0.0,0.080
cotton,0.0,0.060
copra,0.0,0.070
fats,0.0,0.050
fish,0.0,0.070
fishlq,0.0,0.070


In [ ]:
ingredients_plus_bounds = pd.merge(left=ingredients, right=ingredient_bounds_df, left_on='Reference name', right_on=ingredient_bounds_df.index, how='outer')
ingredients_plus_bounds.set_index('Reference name')
ingredients_plus_bounds

,Name,Reference name,Availability,Price,dm,moist,ash,cp,cfat,cfibre,staew,sug,cho,ca,p,ip,k,na,cl,oebr,oelh,opp,lys,met,cys,mc,thr,trp,ile,arg,phe,his,leu,tyr,phetyr,val,dlysp,dmetp,dcysp,dmcp,dthrp,dtryp,dilep,dargp,dvalp,dneaa,c182,c20,ne2015mj,ne2015kcal,ewcvb2015,vppi,sidlys,sidmet,sidcys,sidmc,sidthr,sidtrp,sidile,sidneaa,decat,pcat,alyscat,ametcat,acyscat,amccat,athrcat,atrpcat,ailecat,aargcat,aphecat,ahiscat,aleucat,atyrcat,aptcat,avalcat,Lower Bound,Upper Bound
0,Barley,barley,True,0.26,867.0,133.0,19.9,99.7,27.0,43.4,539.3,22.5,0.0,0.4,3.1,2.3,4.9,0.1,1.0,2656.5,2862.0,1.2,3.6,1.7,2.2,3.9,3.4,1.2,3.5,4.9,0.0,0.0,0.0,0.0,0.0,4.9,2.3,1.3,1.6,2.9,2.3,0.9,2.5,3.9,3.6,41.0,6.8,0.0,10.0,2389.5,1.1,1.0,2.7,1.4,1.7,3.1,2.7,1.0,2.9,44.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.100
1,Blood meal 80% CP,blood,True,0.71,890.0,110.0,25.0,800.0,10.0,10.0,0.0,0.0,80.0,0.5,1.7,0.0,2.8,5.8,3.4,2450.0,2450.0,1.3,71.2,9.6,9.6,19.2,35.2,12.0,9.6,34.4,55.2,51.2,102.5,23.2,78.5,68.8,51.3,6.9,6.9,13.8,25.4,8.7,6.9,24.8,49.6,239.8,0.0,0.0,7.2,1716.4,0.8,1.0,60.0,7.6,7.6,15.3,28.0,9.8,6.5,263.6,3690.0,1.0,59.6,7.5,7.4,14.9,27.6,9.7,6.2,29.1,45.7,43.3,85.8,18.2,63.9,57.0,0.0,0.030
2,Blood meal 40% CP,bloodlq,False,0.55,930.0,70.0,500.0,400.0,10.0,0.0,0.0,0.0,20.0,1.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35.6,4.8,4.8,9.6,17.6,6.0,4.8,17.2,27.6,25.6,51.2,11.6,39.2,34.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1527.9,1.5,24.2,3.1,3.0,6.1,11.2,3.9,2.5,11.8,18.5,17.6,34.8,7.4,25.9,26.3,NaN,NaN
3,Bone ash,boneash,True,0.18,954.0,46.0,806.1,0.0,0.0,0.0,0.0,0.0,0.0,289.3,133.8,0.0,0.4,9.5,0.0,0.0,0.0,82.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,87.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,107.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.030
4,Brewers grain (dried),brew,False,0.08,915.0,85.0,46.0,248.0,70.0,132.0,40.0,9.0,551.0,3.5,4.6,3.0,0.5,0.1,0.1,0.0,0.0,0.0,9.4,4.7,4.7,9.4,9.2,3.0,9.9,12.1,12.4,5.7,19.6,7.7,20.1,13.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1340.7,2.8,5.3,2.9,2.5,5.4,5.1,1.7,6.0,8.1,7.9,3.4,12.0,5.1,13.0,7.9,NaN,NaN
5,Copra,copra,False,0.16,907.0,93.0,60.8,203.2,89.0,113.4,13.6,100.7,0.0,0.8,5.5,2.7,21.2,0.6,6.2,1889.0,1889.3,2.6,5.1,3.1,3.1,6.2,6.1,1.5,6.5,22.2,0.0,0.0,0.0,0.0,0.0,9.8,2.4,2.4,1.6,4.1,3.6,1.0,4.9,19.0,7.5,60.0,1.3,0.0,9.1,2163.2,1.0,1.3,2.9,1.8,1.8,3.6,3.5,0.8,3.8,49.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.070
6,Cotton seed cake,cotton,True,0.20,933.0,67.0,59.7,362.9,79.0,169.8,33.6,38.3,0.0,2.2,10.2,7.7,14.4,0.0,0.3,1730.0,1890.3,3.1,14.9,5.8,6.2,11.9,11.7,4.4,11.3,38.8,0.0,0.0,0.0,0.0,0.0,16.0,8.8,3.9,4.2,8.1,8.0,3.2,8.1,32.7,11.8,125.4,28.4,0.6,6.8,1633.7,0.8,2.9,10.4,4.7,4.9,9.6,8.9,3.5,9.0,139.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.060
7,Fats/ Vegetable oil,fats,False,1.50,995.0,5.0,0.0,0.0,995.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7230.0,9770.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,500.0,35.0,31.8,7600.0,3.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8785.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.050
8,Fish meal 56% CP,fish,True,0.63,911.0,89.0,247.0,563.0,90.0,0.0,0.0,0.0,11.0,60.0,39.6,0.0,6.4,10.5,15.2,2862.0,2982.0,19.6,42.8,15.8,5.1,20.9,23.6,6.2,23.6,33.2,22.0,14.6,41.1,17.4,36.4,27.6,38.5,14.3,4.5,18.8,20.1,5.6,21.0,30.6,32.3,211.2,1.3,56.9,8.8,2099.0,1.0,30.5,38.2,14.0,3.7,17.7,20.9,5.3,21.3,210.8,3501.0,23.8,36.4,13.3,4.2,17.5,19.6,5.0,19.6,28.8,18.3,12.1,34.5,14.3,32.6,22.7,0.0,0.070
9,Fish meal 40% CP,fishlq,True,0.55,900.0,100.0,399.0,384.0,100.0,0.0,0.0,0.0,18.0,141.0,67.0,0.0,6.4,10.5,15.2,0.0,0.0,0.0,24.8,7.6,2.6,10.2,12.2,2.8,9.9,20.8,10.6,6.0,17.9,7.1,17.8,12.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,45.0,0.0,0.0,0.0,0.

In [ ]:
available_ingredients_plus_bounds = ingredients_plus_bounds[ingredients_plus_bounds.Availability][['Name','Reference name','Price','Upper Bound','Lower Bound']+list(nutrient_rules.Nutrient)].set_index('Reference name')
available_ingredients_plus_bounds['Upper Bound'].replace(np.nan, np.inf, inplace=True)
available_ingredients_plus_bounds['Lower Bound'].replace(np.nan, 0, inplace=True)
available_ingredients_plus_bounds

,Name,Price,Upper Bound,Lower Bound,oebr,cp,cfibre,staew,ca,na,opp,dlysp,dmetp,dmcp,dthrp,dtryp,dvalp,dargp
Reference name,,,,,,,,,,,,,,,,,,
barley,Barley,0.26,0.100,0.0,2656.5,99.7,43.4,539.3,0.4,0.1,1.2,2.3,1.3,2.9,2.3,0.9,3.6,3.9
blood,Blood meal 80% CP,0.71,0.030,0.0,2450.0,800.0,10.0,0.0,0.5,5.8,1.3,51.3,6.9,13.8,25.4,8.7,49.6,24.8
boneash,Bone ash,0.18,0.030,0.0,0.0,0.0,0.0,0.0,289.3,9.5,82.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cotton,Cotton seed cake,0.20,0.060,0.0,1730.0,362.9,169.8,33.6,2.2,0.0,3.1,8.8,3.9,8.1,8.0,3.2,11.8,32.7
fish,Fish meal 56% CP,0.63,0.070,0.0,2862.0,563.0,0.0,0.0,60.0,10.5,19.6,38.5,14.3,18.8,20.1,5.6,32.3,30.6
fishlq,Fish meal 40% CP,0.55,0.070,0.0,0.0,384.0,0.0,0.0,141.0,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
gnseeds,Groundnut seeds,0.59,0.100,0.0,5243.0,287.1,23.3,0.0,1.0,0.0,1.7,7.5,2.8,5.9,5.8,2.4,9.5,28.1
maize,Maize/ Corn,0.33,inf,0.2,3227.8,76.3,20.8,648.5,0.1,0.0,0.7,1.4,1.4,2.7,2.1,0.4,2.9,3.1
maizebranhighq,"Maize bran, high quality",0.19,0.250,0.0,2893.4,90.6,39.6,489.3,1.3,0.2,1.4,1.8,1.6,3.1,2.4,0.5,3.8,3.9


In [ ]:
nutrient_bounds = { nut : (lb if not pd.isna(lb) else 0,     # no lower bound translates into 0 as lower bound
                           ub if not pd.isna(ub) else None)  # no upper bound becomes None
                    for nut,lb,ub in zip(nutrient_rules.Nutrient,nutrient_rules['Lower Bound'],nutrient_rules['Upper Bound']) 
                  }

In [ ]:
nutrient_bounds_df = pd.DataFrame.from_dict(nutrient_bounds, orient='index')
nutrient_bounds_df = nutrient_bounds_df.set_axis(['Lower Bound', 'Upper Bound'], axis=1)
nutrient_bounds_df.replace(np.nan, np.inf, inplace=True)
nutrient_bounds_df

,Lower Bound,Upper Bound
oebr,2750.0,2850.0
cp,155.0,195.0
cfibre,0.0,75.0
staew,300.0,inf
ca,7.0,9.0
na,1.4,2.1
opp,3.5,inf
dlysp,6.7,7.9
dmetp,2.8,inf
dmcp,5.1,6.1


In [ ]:
combined_ingredient_rules   = [] 
set_of_availabe_ingredients = set(available_ingredients_plus_bounds.index)
for c in ['Unnamed: '+str(i) for i in range(5,13)]:
    aux = ingredient_rules[[c]].dropna().values
    aux = [ v[0] for v in aux ]
    upperbound          = aux[0]
    ingredients_in_rule = set(aux[1:]).intersection(set_of_availabe_ingredients)
    if ingredients_in_rule:
        combined_ingredient_rules.append((upperbound,ingredients_in_rule))
combined_ingredient_rules

[(0.17, {'cotton', 'sunflower'}),
 (0.35, {'soybeanexp', 'soybeanmeal'}),
 (0.3, {'barley'}),
 (0.25, {'maizebranhighq', 'maizebranlowq'}),
 (0.07, {'fish', 'fishlq'}),
 (0.08, {'mbmeal'}),
 (0.25, {'casfine', 'caswhole', 'tapbran'})]

In [ ]:
df = pd.DataFrame (combined_ingredient_rules, columns = ['Restriction', 'Ingredients'])
df

,Restriction,Ingredients
0,0.17,"{cotton, sunflower}"
1,0.35,"{soybeanmeal, soybeanexp}"
2,0.30,{barley}
3,0.25,"{maizebranhighq, maizebranlowq}"
4,0.07,"{fishlq, fish}"
5,0.08,{mbmeal}
6,0.25,"{casfine, tapbran, caswhole}"


In [ ]:
d = [pd.DataFrame(df['Ingredients'].tolist()).add_prefix('Ingredient')]
df1 = pd.concat(d, axis=1)#df1 is the dataframe that has only the names of the ingredients for which we have combined rules 
df1.replace([None], 0, inplace=True)
df2 = df1.join(df)#combine 2 dataframes 
df2 = df2.drop(columns="Ingredients")#remove the column 'ingredients' from the df2
df2



,Ingredient0,Ingredient1,Ingredient2,Restriction
0,cotton,sunflower,0,0.17
1,soybeanmeal,soybeanexp,0,0.35
2,barley,0,0,0.30
3,maizebranhighq,maizebranlowq,0,0.25
4,fishlq,fish,0,0.07
5,mbmeal,0,0,0.08
6,casfine,tapbran,caswhole,0.25


In [ ]:
def CreateFeedCalculatorModel(available_ingredients_plus_bounds, nutrient_bounds_df):

 model          = pyo.ConcreteModel('Feed Calculator')#create model
 model.ingredients = pyo.Set( initialize = available_ingredients_plus_bounds.index )#create set of ingredients(names)
 model.nutrients = pyo.Set( initialize = nutrient_bounds_df.index )#create set of nutrients(names)

 model.n = pyo.Var( model.ingredients, within=pyo.NonNegativeReals )#create the variables, model.n-amount of certain ingredient

 model.expenses = pyo.Objective(expr = pyo.quicksum(available_ingredients_plus_bounds.Price[i]*model.n[i] for i in model.n)
                          , sense=pyo.minimize)#set the objective:minimize the sum of the costs of all the ingredients

 

 def lower_bound_ing(model,i):#function: restrition about lower bound of the ingredient
  return  model.n[i] >= available_ingredients_plus_bounds['Lower Bound'][i]
 model.lower_bound_ing = pyo.Constraint( model.ingredients, rule=lower_bound_ing)



 def upper_bound_ing(model,i):#function: restrition about upper bound of the ingredient
   return model.n[i]<=available_ingredients_plus_bounds['Upper Bound'][i]
 model.upper_bound_ing = pyo.Constraint( model.ingredients, rule=upper_bound_ing)
 
 model.overall_weight = pyo.Constraint(expr = pyo.quicksum( model.n[i] for i in model.ingredients )==1) #the sum of all the ingredients should be 1 kg

 model.lower_bound_nutrients = pyo.ConstraintList()#restriction about the lower bound of the nutrient
 for j in model.nutrients:
   model.lower_bound_nutrients.add(pyo.quicksum([available_ingredients_plus_bounds.loc[i][j]*model.n[i]for i in model.ingredients]) >= nutrient_bounds_df['Lower Bound'][j])

 model.upper_bound_nutrients = pyo.ConstraintList()#restriction about the upper bound of the nutrient
 for j in model.nutrients:
   model.upper_bound_nutrients.add(pyo.quicksum([available_ingredients_plus_bounds.loc[i][j]*model.n[i]for i in model.ingredients]) <= nutrient_bounds_df['Upper Bound'][j])

 model.upper_bound_combined = pyo.ConstraintList()#combined rules
 for g in range (df1.shape[0]):#go through every line of data frame woth the names of all the ingredients from the combined rules 
   l=[]#create empty list, where we will put the names of ingredients for which do we have a restriction
   for h in range(df1.shape[1]):#go throug every column of the ingredients from the df
     if df1.iloc[g][h]!=0:#take only ingredients with rules not empty cells
      l.append(df1.iloc[g][h])#add ingredient for which do we have rectriction to the list
   model.ingredients_subset = pyo.Set( initialize = l )#make a set from the ingredients of a certain restriction
   model.upper_bound_combined.add(pyo.quicksum([model.n[t] for t in model.ingredients_subset]) <= df2.loc[g]['Restriction'])
    
 return model

In [ ]:
m = CreateFeedCalculatorModel(available_ingredients_plus_bounds, nutrient_bounds_df)
m.pprint()

    (type=<class 'pyomo.core.base.set.OrderedScalarSet'>) on block Feed
    Calculator with a new Component (type=<class
    'pyomo.core.base.set.AbstractOrderedScalarSet'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    (type=<class 'pyomo.core.base.set.OrderedScalarSet'>) on block Feed
    Calculator with a new Component (type=<class
    'pyomo.core.base.set.AbstractOrderedScalarSet'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    (type=<class 'pyomo.core.base.set.OrderedScalarSet'>) on block Feed
    Calculator with a new Component (type=<class
    'pyomo.core.base.set.AbstractOrderedScalarSet'>). This is usually
    indicative of a modelling error. To avoid this warning, use
    block.del_component() and block.add_component().
    (type=<class 'pyomo.core.base.set.OrderedScalarSet'>) on block Feed
    

In [ ]:
%time results = pyo.SolverFactory('cbc').solve(m)
print(results.solver.status, results.solver.termination_condition )

m.display()



CPU times: user 40.9 ms, sys: 11.7 ms, total: 52.6 ms
Wall time: 99.2 ms
ok optimal
Model Feed Calculator

  Variables:
    n : Size=26, Index=ingredients
        Key            : Lower : Value         : Upper : Fixed : Stale : Domain
                barley :     0 :           0.0 :  None : False : False : NonNegativeReals
                 blood :     0 :           0.0 :  None : False : False : NonNegativeReals
               boneash :     0 :           0.0 :  None : False : False : NonNegativeReals
               casfine :     0 :           0.0 :  None : False : False : NonNegativeReals
              caswhole :     0 :    0.18676756 :  None : False : False : NonNegativeReals
                cotton :     0 :          0.05 :  None : False : False : NonNegativeReals
                dicaph :     0 : 0.00052158349 :  None : False : False : NonNegativeReals
                    dl :     0 : 0.00073393208 :  None : False : False : NonNegativeReals
                  fish :     0 :   0.01412632

In [ ]:
print("Total costs: $" + str(pyo.value(m.expenses)))
solution = pd.Series( [pyo.value( m.n[i]*1000 ) for i in m.ingredients ], index = m.ingredients)#receive the dataframe with all the answers; *1000 ti receive the answer in grams
solution

Total costs: $0.2697643896412


ingredients
barley              0.000000
blood               0.000000
boneash             0.000000
cotton             50.000000
fish               14.126323
fishlq              0.000000
gnseeds            55.750927
maize             200.000000
maizebranhighq    250.000000
maizebranlowq       0.000000
mbmeal             50.000000
sugars              0.000000
soybeanexp         68.359545
soybeanmeal         0.000000
sunflower         120.000000
sunflowerseeds      0.000000
tapbran             0.000000
caswhole          186.767560
casfine             0.000000
wheatbran           0.000000
lysine              1.555065
dl                  0.733932
ltryp               0.000000
dicaph              0.521583
shells              0.000000
salt                2.185066
dtype: float64